In [343]:
import re, os
import pathlib as pl
import sys

In [2]:
home = pl.Path(os.getcwd()).parent

In [348]:
sys.path.append(str(home))

In [352]:
from src.core import *

In [353]:
inputs = home/'data'/'input'
outputs = home/'data'/'output'

In [13]:
gage_table = pd.read_excel(inputs/'gages.xlsx', 'BasinChar_all_HUCs_Final')

In [14]:
#missing Atlas 14 data
gage_table.loc[gage_table['NOAA_2yr24hr_in'] <0]

,OBJECTID,HUC8,GID,Station_Name,TDA_SqMi,NCDA_SqMi,CDA_SqMi,BP_Mi,BL_Mi,BW_Mi,...,PRISM_yr_mm,NOAA_2yr24hr_in,NOAA_10yr24hr_in,CN,KSAT_mimps,USFWS_Pct_Wetlands,USFWS_Pct_LakePond,MeanEVI,MeanNDVI,B17C
0,1,10120105,6382200,"PRITCHARD DRAW NEAR LANCE CREEK, WY",5.295270,0.000000,5.295270,12.112100,3.335270,1.587659,...,361.872192,-9.999,-9.999,82,2.527000,0.109145,0.111923,0.185806,0.322918,Completed
29,30,10150002,6454000,NIOBRARA RIVER AT WYO-NEBR. STATE LINE,462.623993,0.000000,462.623993,152.891998,33.619400,13.760626,...,413.893250,-9.999,-9.999,70,18.785999,0.528039,0.037628,0.297196,0.483519,Completed
30,28,10150002,6454100,"Niobrara River at Agate, Nebr.",938.544983,16.636499,921.908997,232.722000,52.642502,17.512636,...,404.515686,-9.999,-9.999,69,27.069000,0.363433,0.024104,0.277967,0.453896,Completed
79,63,10180008,6655000,"COTTONWOOD CREEK AT WENDOVER, WY",194.425995,0.000000,194.425995,110.610001,30.385799,6.398581,...,384.281738,-9.999,-9.999,72,11.183000,0.365822,0.044322,0.225011,0.382250,Need B17C
80,66,10180012,6675300,"HORSE CREEK TRIBUTARY NEAR LITTLE BEAR, WY",8.147290,0.000000,8.147290,17.418100,5.567540,1.463355,...,390.494293,-9.999,-9.999,71,9.797000,0.574323,0.040936,0.357875,0.591165,Maybe
81,67,10180012,6677500,"HORSE CREEK NEAR LYMAN, NE",1723.010010,0.000000,1723.010010,362.441986,87.784897,19.627636,...,412.831085,-9.999,-9.999,73,23.363001,0.779456,0.243305,0.290406,0.447816,Completed
88,72,10190015,6762500,"Lodgepole Creek at Bushnell, Nebr.",1215.719971,0.000000,1215.719971,278.145996,80.838203,15.038929,...,405.837433,-9.999,-9.999,76,9.704000,0.389473,0.047446,0.262907,0.423975,Completed
89,74,10190016,6762600,"LODGEPOLE CREEK TRIBUTARY NO 2 NEAR ALBIN, WY",7.731770,0.000000,7.731770,17.859501,6.125790,1.262167,...,442.665131,-9.999,-9.999,81,7.082000,0.086981,0.005508,0.274632,0.377728,Completed


In [15]:
#Bulletin 17C cross reference

In [16]:
import glob

In [17]:
rpts = glob.glob(str(inputs)+'/**/*.rpt', recursive=True)

In [18]:
len(rpts)

665

In [19]:
def find_link(x,rpts):
    out = ""
    for lnk in rpts:
        if lnk.find(str(x)) >= 0:
            if out != '':
                out+='; '
            out+=lnk
    if len(out) == 0:
        return None
    else:
        return out

In [20]:
gage_table['17C_link_rpt'] = gage_table['GID'].apply(lambda x: find_link(x,rpts))

In [17]:
gage_table.to_excel(outputs/'out_analysis.xlsx')

In [21]:
rpt_test = gage_table.iloc[0]['17C_link_rpt']

In [34]:
rpt = rpt_test.split('; ')[-1]

In [35]:
str(rpt)

'C:\\git\\flow-regression-analysis\\data\\input\\dss\\GagesForStantec\\6382200\\Bulletin17Results\\06382200\\06382200.rpt'

In [341]:
rpt_to_pd(rpt)

,"Computed Curve FLOW, CFS",Variance Log(EMA),Percent Chance Exceedance,"Confidence Limits 0.05 FLOW, CFS","Confidence Limits 0.95 FLOW, CFS"
0,"9,322.6",0.050454,0.2,"30,737.3","4,730.1"
1,"7,456.6",0.041325,0.5,"21,647.7","4,003.9"
2,"6,147.2",0.034899,1.0,"16,167.1","3,447.1"
3,"4,931.9",0.028934,2.0,"11,723.4","2,887.0"
4,"3,816.1",0.023495,4.0,"8,187.8","2,326.8"
5,"2,503.0",0.017332,10.0,"4,669.3","1,598.6"
6,"1,638.9",0.013844,20.0,"2,753.3","1,066.9"
7,674.3,0.013206,50.0,"1,035.5",413.0
8,249.4,0.021913,80.0,407.0,122.6
9,141.9,0.031551,90.0,249.7,58.2
